In [4]:
%load_ext autoreload
%autoreload 2
from dolfinx import fem, mesh, plot, nls, log
from dolfinx.io import gmshio
from mpi4py import MPI
import ufl
from petsc4py import PETSc
from slepc4py import SLEPc
import numpy as np
from dolfinx.io import XDMFFile
import matplotlib.pyplot as plt
from scipy.linalg import null_space
from dolfinx_utils import *
from helmholtz_x.passive_flame_x import *
from helmholtz_x.eigensolvers_x import eps_solver
from helmholtz_x.eigenvectors_x import normalize_eigenvector, normalize_unit
from helmholtz_x.dolfinx_utils import XDMFReader, xdmf_writer 
import pyvista

%autoreload utils.dolfin_utils

In [5]:
N = 10
degree = 1
c_const = 1
msh = mesh.create_unit_square(MPI.COMM_WORLD, N, N)
c = fem.Constant(msh, PETSc.ScalarType(c_const))


In [6]:
boundaries = [(1, lambda x: np.isclose(x[0], 0)),
              (2, lambda x: np.isclose(x[0], 1)),
              (3, lambda x: np.isclose(x[1], 0)),
              (4, lambda x: np.isclose(x[1], 1))]
    
ds, facet_tags = tag_boundaries(msh, boundaries=boundaries, return_tags=True)

boundary_conditions = {4: {'Dirichlet'},
                       3: {'Dirichlet'},
                       2: {'Dirichlet'},
                       1: {'Dirichlet'}}

In [7]:
V = fem.FunctionSpace(msh, ("CG", 3))

#Boundary Conditions
bound_func = fem.Function(V)
bound_func.interpolate(lambda x: x[0]*0)

def onbound(x):
    return np.logical_or(np.logical_or(np.isclose(x[0], 0), np.isclose(x[0], 1)), 
                                np.logical_or(np.isclose(x[1], 1), np.isclose(x[1], 0)))

bound_facets = mesh.locate_entities_boundary(msh, msh.topology.dim-1, onbound)
bound_dof = fem.locate_dofs_topological(V, msh.topology.dim-1, bound_facets)

bound_cond = fem.dirichletbc(bound_func, bound_dof)

In [8]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
f = fem.Constant(msh, PETSc.ScalarType(1))

a = ufl.inner(ufl.grad(u), ufl.grad(v))*ufl.dx
L = ufl.inner(f, v)*ufl.dx

problem = fem.petsc.LinearProblem(a, L, petsc_options={"ksp_type": "preonly", "pc_type": "lu"}, bcs=[bound_cond])
b = problem.solve()

In [9]:
v = ufl.TestFunction(V)
f = fem.Constant(msh, PETSc.ScalarType(1))

a = ufl.inner(ufl.sqrt(ufl.dot(ufl.grad(b), ufl.grad(b))), v)*ufl.dx - ufl.inner(f, v)*ufl.dx

problem = fem.petsc.NonlinearProblem(a, b, bcs=[bound_cond])

In [10]:
solver = nls.petsc.NewtonSolver(MPI.COMM_WORLD, problem)
solver.convergence_criterion = "incremental"
solver.rtol = 1e-6
solver.report = True

ksp = solver.krylov_solver
opts = PETSc.Options()
option_prefix = ksp.getOptionsPrefix()
opts[f"{option_prefix}ksp_type"] = "cg"
opts[f"{option_prefix}pc_type"] = "gamg"
opts[f"{option_prefix}pc_factor_mat_solver_type"] = "mumps"
ksp.setFromOptions()

log.set_log_level(log.LogLevel.INFO)
n, converged = solver.solve(b)
assert(converged)
print(f"Number of interations: {n:d}")

2022-11-08 11:34:08.447 (  10.258s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-11-08 11:34:14.973 (  16.784s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-11-08 11:34:21.491 (  23.302s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 2: r (abs) = 29.9137 (tol = 1e-10) r (rel) = 1.24652(tol = 1e-06)
2022-11-08 11:34:21.494 (  23.305s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-11-08 11:34:21.505 (  23.315s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 3: r (abs) = 1389.84 (tol = 1e-10) r (rel) = 57.9155(tol = 1e-06)
2022-11-08 11:34:21.508 (  23.318s) [main            ]              petsc.cpp:677   INFO| PETSc Krylov solver starting to solve system.
2022-11-08 11:34:23.631 (  25.441s) [main            ]       NewtonSolver.cpp:36    INFO| Newton iteration 4: 

RuntimeError: Newton solver did not converge because maximum number of iterations reached

In [ ]:
plotter = pyvista.Plotter()

topology, cell_types, geometry = plot.create_vtk_mesh(msh, msh.topology.dim)
grid = pyvista.UnstructuredGrid(topology, cell_types, geometry)
plotter.add_mesh(grid, show_edges=True)

u_topology, u_cell_types, u_geometry = plot.create_vtk_mesh(V)
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry)

u_grid.point_data["b"] = b.x.array.real
u_grid.set_active_scalars("b")
plotter.add_mesh(u_grid, show_edges=True)

plotter.view_xy()
plotter.show()

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [14]:
np.max(b.vector[:])

(0.07366554903923167+0j)